<a href="https://colab.research.google.com/github/JammieBurger/imlo-project-actual/blob/main/IMLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialising Training and Testing Data.

In [27]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
from torchvision.transforms import ToTensor
import torch.nn.functional as F

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
trainingData = datasets.Flowers102(
    root="data",
    split='train',
    transform=transform,
    download=True)
testData = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform=transform)

Creating Neural Network

In [ ]:
batchSize = 128

trainDataloader = DataLoader(trainingData,batchSize)
testDataloader = DataLoader(testData,batchSize)
device = ("cuda"if torch.cuda.is_available()
                else "mps"
                if torch.backends.mps.is_available()
                else "cpu")
print(f"Using {device} device")
class NeuralNetwork(nn.Module):
    def __init__(self,inputFeatures=224*224*3,layerOne=2,layerTwo=3,outputFeatures=512):
        super().__init__()
        self.flatten = nn.Flatten()
        self.firstLayer = nn.Linear(inputFeatures,layerOne)
        self.secondLayer = nn.Linear(layerOne,layerTwo)
        self.finalLayer = nn.Linear(layerTwo,outputFeatures)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.firstLayer(x))
        x = F.relu(self.secondLayer(x))
        x = self.finalLayer(x)
        return x

Creating Training and Testing Functions

In [ ]:
model = NeuralNetwork().to(device)
#print(model)
lossFunction = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = lossFunction(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    numBatches = len(dataloader)
    model.eval()
    testLoss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            testLoss += lossFunction(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    testLoss /= numBatches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {testLoss:>8f} \n")

Going through the Epochs

In [ ]:
epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainDataloader, model, lossFunction, optimizer)
    test(testDataloader, model, lossFunction)
print("Done!")
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))